Project 1

# Project planning info and thoughts

"Predicting card price and power as a factor of rarity, card type (creature, artifact, etc), creature type (wizard, elf, etc), abilities (flying vs draw a card vs trample vs indestructible etc), card color, mana cost (maybe affected by rarity?)"

* Interesting context: pre- and post-MTGA. Esport affect on what makes cards valuable? // Pandemic?
   * Reflects changing mtg design?

* speculative pricing - lithoform engine etc
   * could cross-ref with standard metagame? (mtggoldfish, mtgarena data? Like, why are these cards expensive when they don't actually see play?)

* Predict format prices? Standard vs Pioneer vs Modern? How much does this hobby cost?
   * How do relationships among variables differ among formats? (Standard, pioneer, modern)
   * Late stage - remove banned cards?
    
* Compare across websites: CK (a centralized company) vs TCGplayer or ebay (which are aggregations of individual sellers) 
   * which is better

Data sources for magic:  
* EDHREC  
* scryfall  
* Card kingdom  
* TCGplayer  
* Channel fireball  
* SCGames  
* Gatherer? (no prices)  
* youtube? (this might be difficult)  
* MTGGoldfish  
* Arena data?  

X (independent/feature variables):
* Rarity - dummy var
    * rarity_M
    * rarity_R
    * rarity_U
    * rarity_C
* Card color - dummy var
    * is_w
    * is_u
    * is_b
    * is_r
    * is_g
* Number of lines of rules text
* Converted mana cost
* Time since set release? 
    * most applicable for Pioneer and modern datasets

* Is it multicolored? or Number of colors
* Card type
* Creature type
* Power/toughness
* Abilities/number of abilities (could assign points for each such that a total "ability score" could be generated)
    * Flying
    * Trample
    * Indestructible
    * Hexproof
* Artist
* Number of existing prints -- challenging in standard (but idea to exclude "( )" - showcase etc)

y (dependendent/target variable):
* price CK
* perhaps add prices for other websites


# Getting data

In [724]:
#   Importing libraries
from bs4 import BeautifulSoup
import re   #regular expressions
import requests
import numpy as np
import pandas as pd
import seaborn as sns

In [374]:
# Sending request to Card Kingdom

modern_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2864&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any' 

standard_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2779&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any'

response = requests.get(standard_url)


In [27]:
response.status_code  #200 = success!

200

In [28]:
response.text[:1000]  #First 1000 characters of the HTML

'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta http-equiv="Cache-control" content="max-age=29030400,public">\n<meta name="description" content="Magic The Gathering, magic cards, singles, decks, card lists, deck ideas, wizard of the coast, all of the cards you need at great prices are available at Cardkingdom.">\n<meta name="keywords" content="Magic The Gathering, magic cards, magic the gathering and purchase, magic singles, magic the gathering pricelist, magic the gathering card lists, magic the gathering deck ideas, magic the gathering cards, magic the gathering cards sell, magic the gathering card prices, wizard of the coast, magic the gathering singles, magic the gathering decks, magic the gathering cardlist, cheap magic singles.">\n<meta NAME="robots" CONTENT="noindex, follow">\n<meta name="google-site-verification" content="mG_HnQSIpvWtvlgoloS5nWbuB7wNauY3RlU6dEmTLec" />\n<meta http-equiv="X-UA-Compatible" c

## Creating soup object

This is redone in all_scrape() function below

In [375]:
page = response.text
soup = BeautifulSoup(page)

In [ ]:
print(soup.prettify)

## Functions to scrape variables

Depends on soup object

In [458]:
# Getting name
def get_name(soup):
    name = [link.text.strip() for link in soup.find_all('span', class_ = 'productDetailTitle')]
    return name

# get_name()[:5]
# len(name)

In [459]:
# Getting expansion and rarity

expansion_rarity_long = [link.text for link in soup.find_all('div', class_ = 'productDetailSet')]
expansion_rarity_long

def get_expansion_rarity_clean(soup):
    expansion_rarity_long = [link.text for link in soup.find_all('div', class_ = 'productDetailSet')]
    expansion_list = []
    rarity_list = []
    for expansion in expansion_rarity_long:
#         print(expansion)
        expansion_and_rarity = expansion.strip()  # trimming whitespace
        
        expansion_short = expansion_and_rarity[:-4]  # getting only expansion
        expansion_list.append(expansion_short)

        rarity = re.findall("[^()]", expansion_and_rarity[-3:])  # getting only rarity
        rarity_list.append(rarity[0]) # re.findall returns list of lists, so want only first element
        
    return expansion_list, rarity_list

# print(get_expansion_rarity_clean())

# expansion, rarity = get_expansion_rarity_clean()

# print(expansion[:5])

# print('\n')

# print(rarity[:5])


In [502]:
# Getting price

# price_long = soup.findAll("span", attrs = {"class": "stylePrice"})

def get_price_clean(soup):
    price_long = soup.findAll("span", attrs = {"class": "stylePrice"})
    price_list = list()
    for price in price_long:
        x = price.text.strip()
        y = re.findall('\$(\d+.\d+)',x)
        conv_price = float(y[0])
        #print(conv_price)
        price_list.append(conv_price)
    return price_list[::4]

# get_price_clean()[:5]

# price = get_price_clean()

# price = get_price_clean() # CK includes prices for 4 card gradings, so only taking NM

# price

In [461]:
# Getting full written-out costs

def get_mana_costs(soup):
    all_mana_costs_long = soup.findAll("div", attrs = {"class": "productDetailCastCost"})  # find list of card CMC's as sets of one or more mana symbol images

    cost = []
    for card_cost in all_mana_costs_long:  # iterate through list of all card CMC pic sets
        CMC_list = []
        for mana_symbol in card_cost.find_all('img'): # further iterate through all individual casting cost images (1, g, u, etc)
            CMC_list.append(re.findall('\w+', mana_symbol.get('src'))[4][-1]) # Add to individual card's casting cost
        CMC_str=''
        cost.append(CMC_str.join(CMC_list)) # turn casting cost from list to string
    return cost

# cost = get_mana_costs()

# cost

In [462]:
def get_converted_cost(soup):
    # references get_mana_costs function
     
    cmc = []

    for card_cost in get_mana_costs(soup):
        card_cmc=0
        for letter in card_cost:
            if letter.isdigit():
                card_cmc += int(letter)
            else: 
                card_cmc+=1
        cmc.append(card_cmc)

    return cmc

# get_converted_cost()

In [463]:
def get_card_color(soup):
    color = []
    for card_cost in get_mana_costs(soup):
        card_color=[]
        for letter in card_cost:
            if letter != 'x':
                if not letter.isdigit():
                    card_color.append(letter)
                
        if card_color:
            color.append("".join(set(card_color)))
        else: color.append('colorless')
    return color

# color = get_card_color()

# color

In [613]:
# get rules text
def get_rules_text(soup):
    all_rules_text_long = soup.findAll("tr", attrs = {"class": "detailFlavortext"})
    
    rules_text = []
    for card_rules in all_rules_text_long:
        rules_text.append(card_rules.td.text.strip().replace('\n', ' '))
    
#     rules_text = rules_text_long.find("tr", attrs = {"class": "detailFlavortext"}).td.text.strip().replace('\n', ' ')

    return rules_text

# /html/body/div[4]/div[3]/div[3]/div[2]/div[1]/div[2]/table/tbody/tr[2]/td

In [619]:
get_rules_text(soup)[1]

', : Copy target activated or triggered ability you control. You may choose new targets for the copy. , : Copy target instant or sorcery spell you control. You may choose new targets for the copy. , : Copy target permanent spell you control. (The copy becomes a token.)'

In [704]:
# # Trying to add tap and mana symbols
# # Tabling this for now

# all_rules_text_long = soup.findAll("tr", attrs = {"class": "detailFlavortext"})
# uro = all_rules_text_long[0]
# lith_eng_rules_text = all_rules_text_long[1]

# lith_eng.td.img.get('src')

# # uro.td.img.get('src')

# # uro.td
# # for img in uro.findAll('img'):
# #     print(img.get('src'))

# lith_eng_rules_text_str = str(lith_eng_rules_text)

# img_list=[]
# for img in lith_eng_rules_text.findAll('img'): # looping through all images in rules text
# #     print(img) # a tag
#     src = img.get('src') # a string
# #     print(src)
#     if re.search('mana_2', src):
#         img_new = '2'
#     elif re.search('mana_3', src):
# #         print(3)
#         img_new = '3'
#     elif re.search('mana_4', src):
# #         print(4)
#         img_new = '4'
#     elif re.search('tap', src):
# #         src.replace(old, new)
# #         print('T')
#         img_new = 't'
# #     print(re.findall('mana', src))

#     img_list.append(img_new)
# # print(lith_eng_rules_text.findAll('img'))
 
# lith_eng_rules_text.td.text.strip().replace('\n', ' ')

# # img_list

    
# len(lith_eng_rules_text.findAll('img'))

    

6

ides: loop throgh, create img_list, replace with elements of img_list in return function re.findall ' ,'

In [605]:
def get_mana_costs(soup):
    all_mana_costs_long = soup.findAll("div", attrs = {"class": "productDetailCastCost"})  # find list of card CMC's as sets of one or more mana symbol images

    cost = []
    for card_cost in all_mana_costs_long:  # iterate through list of all card CMC pic sets
        CMC_list = []
        for mana_symbol in card_cost.find_all('img'): # further iterate through all individual casting cost images (1, g, u, etc)
            CMC_list.append(re.findall('\w+', mana_symbol.get('src'))[4][-1]) # Add to individual card's casting cost
        CMC_str=''
        cost.append(CMC_str.join(CMC_list)) # turn casting cost from list to string
    return cost


'When Uro enters the battlefield, sacrifice it unless it escaped. Whenever Uro enters the battlefield or attacks, you gain 3 life and draw a card, then you may put a land card from your hand onto the battlefield. Escape - , Exile five other cards from your graveyard. (You may cast this card from your graveyard for its escape cost.)'

## Using driver to navigate pages

In [314]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [335]:
driver = webdriver.Chrome(chromedriver)
first_page_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2779&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any'
driver.get(first_page_url)

In [336]:
number_of_pages = int(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[5]/div[2]/ul/li[13]/a").text)
number_of_pages

100

## Scraping algorithm

Depends on above functions - basically the infinity gauntlet (the above functions are the infinity stones)

In [729]:
def all_scrape():
    driver = webdriver.Chrome(chromedriver)
    first_page_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2779&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any'
    url = first_page_url
    driver.get(first_page_url)
    number_of_pages = int(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[5]/div[2]/ul/li[13]/a").text)
    
    name = []
    expansion = []
    rarity = []
    mana_cost = []
    cmc = []
    color_identity = []
    rules_text = []
    price = []
    
    for i in range(number_of_pages):
#     for i in range(3): # 3 pages for now
       
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source)


        name_individual = get_name(soup)
        name.extend(name_individual)
        
        expansion_individual, rarity_individual = get_expansion_rarity_clean(soup)
        expansion.extend(expansion_individual)
        rarity.extend(rarity_individual)
        
        mana_cost_individual = get_mana_costs(soup)
        mana_cost.extend(mana_cost_individual)
        
        cmc_individual = get_converted_cost(soup)
        cmc.extend(cmc_individual)
        
        color_identity_individual = get_card_color(soup)
        color_identity.extend(color_identity_individual)
        
        rules_text_individual = get_rules_text(soup)
        rules_text.extend(rules_text_individual)
        
        price_individual = get_price_clean(soup)
        price.extend(price_individual)
        
        
        url = soup.find('ul', class_ = "pagination").find_all('li')[-1].a.get('href')
#         print(url[-5:])
#         print(name)
    
    return name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price

# all_scrape()

In [730]:
name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price = all_scrape()

# print('Name:', name[:5], '\n', 'Expansion:', expansion[:5], '\n', 'Rarity:', rarity[:5], '\n', \
#       'Mana_cost:', mana_cost[:5], 'cmc:', cmc[:5], '\n', 'color:', color_identity[:5])

In [731]:
len(price)

1983

## Constructing df

In [948]:
# With card names as indeces

data_tuples = list(zip(name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price))

df = pd.DataFrame(data_tuples, columns = ['name','expansion', 'rarity', 'cost', 'cmc', 'color_identity', 'rules_text', 'price'])

# df.set_index('name', drop=True, inplace = True)

df.head()

,name,expansion,rarity,cost,cmc,color_identity,rules_text,price
0,"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",44.99
1,Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",24.99
2,Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,22.99
3,Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",17.99
4,Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,17.99


In [931]:
df.shape

(1983, 7)

In [932]:
df['rules_length'] = df['rules_text'].str.len()

df = df[['expansion', 'rarity', 'cost', 'cmc', 'color_identity', 'rules_text', 'rules_length','price']]

df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,rules_length,price
name,,,,,,,,
"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",332,44.99
Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",268,24.99
Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,188,22.99
Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",210,17.99
Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,301,17.99


# Data cleaning!

In [934]:
df.dtypes

expansion          object
rarity             object
cost               object
cmc                 int64
color_identity     object
rules_text         object
rules_length        int64
price             float64
dtype: object

In [935]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1983 entries, Uro, Titan of Nature's Wrath to Wishful Merfolk
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   expansion       1983 non-null   object 
 1   rarity          1983 non-null   object 
 2   cost            1983 non-null   object 
 3   cmc             1983 non-null   int64  
 4   color_identity  1983 non-null   object 
 5   rules_text      1983 non-null   object 
 6   rules_length    1983 non-null   int64  
 7   price           1983 non-null   float64
dtypes: float64(1), int64(2), object(5)
memory usage: 139.4+ KB


In [936]:
df.describe()

,cmc,rules_length,price
count,1983.000000,1983.000000,1983.000000
mean,2.862330,157.785678,2.119934
std,1.845907,97.373686,5.172132
min,0.000000,5.000000,0.150000
25%,2.000000,88.000000,0.250000
50%,3.000000,143.000000,0.250000
75%,4.000000,228.000000,1.290000
max,9.000000,502.000000,79.990000


In [937]:
df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,rules_length,price
name,,,,,,,,
"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",332,44.99
Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",268,24.99
Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,188,22.99
Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",210,17.99
Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,301,17.99


In [982]:
# replacing blank cells with NaN's
df = df.replace('', 'NaN')

In [981]:
# Getting rid of special rarities (showcase etc) and basic lands (L)

mask = (df['rarity'] != 'L') & (df['rarity'] != 'S')

df = df[mask]

df.drop(['rarity_L', 'rarity_S'], axis=1, inplace = True)

## Removing duplicates: Extended Art, Borderless, Showcase, Godzilla Series

### Extended art

In [956]:
ex_art_mask = ~df['name'].str.contains("Extended Art")

df = df[ex_art_mask] # ~ means 'not'

### Borderless

In [957]:
borderless_mask = ~df['name'].str.contains("Borderless")

df = df[borderless_mask] # ~ means 'not'

### Showcase

In [960]:
showcase_mask = ~df['name'].str.contains("Showcase")

df = df[showcase_mask] # ~ means 'not'

### Godzilla series

In [958]:
godzilla_mask = ~df['name'].str.contains("Godzilla")

df = df[godzilla_mask] # ~ means 'not'

In [962]:
# Did it work?

df.head(50)

,name,expansion,rarity,cost,cmc,color_identity,rules_text,price
0,"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",44.99
1,Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",24.99
2,Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,22.99
3,Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",17.99
4,Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,17.99
5,Nighthawk Scavenger,Zendikar Rising,R,1bb,3,b,"Flying, deathtouch, lifelink Nighthawk Scaveng...",7.99
6,Brazen Borrower,Throne of Eldraine,M,1uu,3,u,Flash Flying Brazen Burrower can block only cr...,19.99
7,Shark Typhoon,Ikoria: Lair of Behemoths,R,5u,6,u,"Whenever you cast a noncreature spell, create ...",15.99
8,Dryad of the Ilysian Grove,Theros Beyond Death,R,2g,3,g,You may play an additional land on each of you...,15.99
9,"Ashaya, Soul of the Wild",Zendikar Rising,M,3gg,5,g,"Ashaya, Soul of the Wild's power and toughness...",14.99


### Setting index back to name

In [963]:
df.set_index('name', drop=True, inplace = True)

df.head()


,expansion,rarity,cost,cmc,color_identity,rules_text,price
name,,,,,,,
"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",44.99
Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",24.99
Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,22.99
Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",17.99
Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,17.99


# Examining data for correlations: prep for regression analysis

## Adding dummy vars for qualitative vars

### Dummy vars for rarity

In [964]:
rarity_dummies = pd.get_dummies(df[['rarity']])

df = pd.concat([df, rarity_dummies], axis=1, sort=False)

df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,rarity_C,rarity_L,rarity_M,rarity_R,rarity_S,rarity_U
name,,,,,,,,,,,,,
"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",44.99,0,0,1,0,0,0
Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",24.99,0,0,1,0,0,0
Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,22.99,0,0,1,0,0,0
Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",17.99,0,0,1,0,0,0
Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,17.99,0,0,1,0,0,0


### Dummy vars for color Identity 

In [965]:
# Dummy var for each different color identity

cid = pd.get_dummies(df[['color_identity']]) #color_identity_dummies

cid.head()

,color_identity_b,color_identity_bn,color_identity_colorless,color_identity_g,color_identity_gb,color_identity_gr,color_identity_gu,color_identity_gub,color_identity_gur,color_identity_r,...,color_identity_wbr,color_identity_wg,color_identity_wgb,color_identity_wgu,color_identity_wgur,color_identity_wr,color_identity_wrgub,color_identity_wu,color_identity_wub,color_identity_wur
name,,,,,,,,,,,,,,,,,,,,,
"Uro, Titan of Nature's Wrath",0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lithoform Engine,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ancient Greenwarden,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Scourge of the Skyclaves,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Agadeem's Awakening,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [966]:
# stripping dummy columns to be only color letters

cid.columns = [re.findall('_[a-z]*', col)[1].replace('_', '') for col in cid.columns]

cid.head()

,b,bn,colorless,g,gb,gr,gu,gub,gur,r,...,wbr,wg,wgb,wgu,wgur,wr,wrgub,wu,wub,wur
name,,,,,,,,,,,,,,,,,,,,,
"Uro, Titan of Nature's Wrath",0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lithoform Engine,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ancient Greenwarden,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Scourge of the Skyclaves,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Agadeem's Awakening,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [972]:
# summing across all dummy columns containing a given color to get the overall score for that color

for color in ['w', 'u', 'b', 'r', 'g']:    
    new_col_name = []
    
    for col in cid.columns:
        if color in col:
            new_col_name.append(col)

    cid['is_'+ color] = cid[new_col_name].sum(axis=1)

cid.head()

,b,bn,colorless,g,gb,gr,gu,gub,gur,r,...,all_w,all_u,all_b,all_r,all_g,is_w,is_u,is_b,is_r,is_g
name,,,,,,,,,,,,,,,,,,,,,
"Uro, Titan of Nature's Wrath",0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,2,0,0,2
Lithoform Engine,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,0
Ancient Greenwarden,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
Scourge of the Skyclaves,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,2,0,0
Agadeem's Awakening,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,2,0,0


In [973]:
# Taking only 5 columns
cid_subset = cid[['is_w', 'is_u', 'is_b', 'is_r', 'is_g']]

In [974]:
cid_subset

,is_w,is_u,is_b,is_r,is_g
name,,,,,
"Uro, Titan of Nature's Wrath",0,2,0,0,2
Lithoform Engine,0,0,0,2,0
Ancient Greenwarden,0,0,0,0,2
Scourge of the Skyclaves,0,0,2,0,0
Agadeem's Awakening,0,0,2,0,0
...,...,...,...,...,...
Plummet,0,0,0,0,2
Scavenging Harpy,0,0,2,0,0
Triton Waverider,0,2,0,0,0


In [978]:
# Adding back to frame

df = pd.concat([df, cid_subset], axis=1, sort=False)

df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,rarity_C,rarity_L,rarity_M,rarity_R,rarity_S,rarity_U,is_w,is_u,is_b,is_r,is_g
name,,,,,,,,,,,,,,,,,,
"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",44.99,0,0,1,0,0,0,0,2,0,0,2
Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",24.99,0,0,1,0,0,0,0,0,0,2,0
Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,22.99,0,0,1,0,0,0,0,0,0,0,2
Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",17.99,0,0,1,0,0,0,0,0,2,0,0
Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,17.99,0,0,1,0,0,0,0,0,2,0,0


In [979]:
df.shape

(1524, 18)

In [987]:
df.columns

Index(['expansion', 'rarity', 'cost', 'cmc', 'color_identity', 'rules_text',
       'price', 'rarity_C', 'rarity_M', 'rarity_R', 'rarity_U', 'is_w', 'is_u',
       'is_b', 'is_r', 'is_g'],
      dtype='object')

In [ ]:
# could rename cols

# df.rename(columns = {'rarity_C':'is_common', 'rarity_u':'is_uncommon','rarity_R':'is_rare', 'rarity_M':'is_mythic'})

In [988]:
df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,rarity_C,rarity_M,rarity_R,rarity_U,is_w,is_u,is_b,is_r,is_g
name,,,,,,,,,,,,,,,,
"Uro, Titan of Nature's Wrath",Theros Beyond Death,M,1gu,3,gu,"When Uro enters the battlefield, sacrifice it ...",44.99,0,1,0,0,0,2,0,0,2
Lithoform Engine,Zendikar Rising,M,4,4,colorless,", : Copy target activated or triggered ability...",24.99,0,1,0,0,0,0,0,2,0
Ancient Greenwarden,Zendikar Rising,M,4gg,6,g,Reach You may play lands from your graveyard. ...,22.99,0,1,0,0,0,0,0,0,2
Scourge of the Skyclaves,Zendikar Rising,M,1b,2,b,"Kicker When you cast this spell, if it was ki...",17.99,0,1,0,0,0,0,2,0,0
Agadeem's Awakening,Zendikar Rising,M,xbbb,4,b,Return from your graveyard to the battlefield ...,17.99,0,1,0,0,0,0,2,0,0


### Saving df with rarity and color dummies as CSV just in case

In [1246]:
# saving as CSV just in case

df.to_csv(r'standard_data_rarity_color_dummies.csv')

Next steps in prepping for regression (see 'intro-to-regression' notebook):

In [989]:
# df.corr

# sns.pairplot(sub_df[['engine-size', 'price']]);
df.corr()

,cmc,price,rarity_C,rarity_M,rarity_R,rarity_U,is_w,is_u,is_b,is_r,is_g
cmc,1.000000,0.133592,-0.192691,0.265336,0.059135,0.002914,0.026095,0.100465,0.078182,-0.152174,0.101506
price,0.133592,1.000000,-0.232144,0.678291,0.054936,-0.182103,-0.047239,0.046035,-0.023639,0.038785,0.084424
rarity_C,-0.192691,-0.232144,1.000000,-0.224760,-0.442503,-0.535408,-0.051842,-0.057007,-0.048462,-0.022105,-0.053664
rarity_M,0.265336,0.678291,-0.224760,1.000000,-0.150480,-0.182073,0.052888,0.059445,0.034223,0.010019,0.062401
rarity_R,0.059135,0.054936,-0.442503,-0.150480,1.000000,-0.358462,0.012761,0.016111,0.017877,0.051135,0.003666
rarity_U,0.002914,-0.182103,-0.535408,-0.182073,-0.358462,1.000000,0.014006,0.012780,0.016163,-0.028790,0.018942
is_w,0.026095,-0.047239,-0.051842,0.052888,0.012761,0.014006,1.000000,-0.175253,-0.157325,-0.248884,-0.185507
is_u,0.100465,0.046035,-0.057007,0.059445,0.016111,0.012780,-0.175253,1.000000,-0.180919,-0.257719,-0.149607
is_b,0.078182,-0.023639,-0.048462,0.034223,0.017877,0.016163,-0.157325,-0.180919,1.000000,-0.272510,-0.154488
is_r,-0.152174,0.038785,-0.022105,0.010019,0.051135,-0.028790,-0.248884,-0.257719,-0.272510,1.000000,-0.269357


In [ ]:
# challenge: making "time_since_release" variable: 
# need to make new series that maps a year (or month) val to 'expansion' series

## Scraping wikipedia to get set release dates

In [995]:
# time_since_release = []

# for exp in expansion:
#     time_since_release.append()

# expansion[:10]

In [ ]:
# time_since_release_dict = {}

# time_since_release_dict['']

Connecting to wikipedia

In [992]:
# scraping wikipedia?

mtg_set_wiki_url = 'https://en.wikipedia.org/wiki/List_of_Magic:_The_Gathering_sets'

response = requests.get(mtg_set_wiki_url)

response.status_code  #200

200

In [994]:
response.text[:1000]  #First 1000 characters of the HTML

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of Magic: The Gathering sets - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"badeefcc-eae3-4306-b1b5-1fb30c8e1180","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Magic:_The_Gathering_sets","wgTitle":"List of Magic: The Gathering sets","wgCurRevisionId":978729560,"wgRevisionId":978729560,"wgArticleId":507735,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with dead external links","Articles with dead external links from January 2020","Articles w

Making soup object

In [ ]:
page = response.text
soup = BeautifulSoup(page)

print(soup.prettify)

Getting table of core sets

### Scraping expansion names and release dates from wikipedia

In [1235]:
# Scraping set and release date: expansions

expansion_names = []
set_dates = []


for set_ in soup.find_all('table', class_ = 'wikitable')[1].find_all('tr')[3:108]: # list of 'tr's goes to 108
    if not set_.td.get('colspan'): # only cycle row tr's have 'colspan' attribute in first 'td' tag
        try: 
            if set_.find('a').text not in expansion_names:
                print(set_.find('a').text)
                expansion_names.append(set_.find('a').text)

            print(set_.find_all('td')[5].text)
            set_dates.append(set_.find_all('td')[5].text)
        except: continue
    



Arabian Nights
December 1993[31]

Antiquities
March 1994[32]

Legends
June 1994[33]

The Dark
August 1994[34]

Fallen Empires
November 1994[35]

Ice Age
June 1995[36]

Homelands
October 1995[37]

Alliances
June 10, 1996[10]

Mirage
October 7, 1996[10]

Visions
February 3, 1997[41]

Weatherlight
June 9, 1997[10]

Tempest
October 13, 1997[10]

Stronghold
March 2, 1998[10]

Exodus
June 15, 1998[10]

Urza's Saga
October 12, 1998[47]

Urza's Legacy
February 15, 1999[10]

Urza's Destiny
June 7, 1999[49]

Mercadian Masques
October 4, 1999[51]

Nemesis
February 14, 2000[53]

Prophecy
June 5, 2000[54]

Invasion
October 2, 2000[55]

Planeshift
February 5, 2001[56]

Apocalypse
June 4, 2001

Odyssey
October 1, 2001

Torment
February 4, 2002[58]

Judgment
May 27, 2002[59]

Onslaught
October 7, 2002[60]

Legions
February 3, 2003

Scourge
May 26, 2003[62]

Mirrodin
October 3, 2003[63]

Darksteel
February 6, 2004[64]

Fifth Dawn
June 4, 2004[66]

Champions of Kamigawa
October 1, 2004[68]

Betrayers of

In [1236]:
print(len(expansion_names), len(set_dates))

78 78


In [1237]:
# Removing [] from release dates

set_dates_new = []
for i in set_dates:
    if '[' in i:
        if i[-5] == '[': # two digits in brackets
            i = i[:-5]
            set_dates_new.append(i)
        else:  # three digits in brackets
            i=i[:-6]
            set_dates_new.append(i)
    else: set_dates_new.append(i)        
set_dates_new[51] = set_dates_new[51][:-5]

len(set_dates_new)

78

Adding unlinked sets

In [1238]:
# Adding unlinked sets

unlinked_sets = ['Guilds of Ravnica', 'Ravnica Allegiance', 'War of the Spark', 'Throne of Eldraine', 'Theros: Beyond Death', 'Ikoria: Lair of Behemoths', 'Zendikar Rising']

expansion_names.extend(unlinked_sets)


unlinked_release_dates = ['October 5, 2018', 'January 25, 2019', 'May 3, 2019', 'October 4, 2019', 'January 24, 2020', 'May 15, 2020', 'September 25, 2020']

set_dates_new.extend(unlinked_release_dates)

In [ ]:
# Scraping set and release date: core sets

expansion_names = []
set_dates = []


for set_ in soup.find_all('table', class_ = 'wikitable')[0].find_all('tr')[3:????]: # list of 'tr's goes to ????
    if not set_.td.get('colspan'): # only cycle row tr's have 'colspan' attribute in first 'td' tag
        try: 
            if set_.find('a').text not in expansion_names:
                print(set_.find('a').text)
                expansion_names.append(set_.find('a').text)

            print(set_.find_all('td')[5].text)
            set_dates.append(set_.find_all('td')[5].text)
        except: continue
    



### Constructing df of expansion names and release dates

In [1239]:
len(set_dates_new), len(expansion_names)

(85, 85)

In [1244]:
sets_and_rel_dates = pd.DataFrame(list(zip(expansion_names, set_dates_new)), columns = ['expansion', 'rel_date'])

sets_and_rel_dates

,expansion,rel_date
0,Arabian Nights,December 1993
1,Antiquities,March 1994
2,Legends,June 1994
3,The Dark,August 1994
4,Fallen Empires,November 1994
...,...,...
80,War of the Spark,"May 3, 2019"
81,Throne of Eldraine,"October 4, 2019"
82,Theros: Beyond Death,"January 24, 2020"
83,Ikoria: Lair of Behemoths,"May 15, 2020"


In [1088]:
#this is the table with core/base sets

soup.find_all('table', class_ = 'wikitable')[0]

Next steps:
* Look at pickling for data storage!!
* get core sets and dates and append to names_and_dates df
* get supp sets and dates and append to names_and_dates df
* append that to larger df
* convert relaease date to time_since_release
* begin running correlation/sns.pairplot()
* repeat for modern and pioneer datasets!!
    * MASSIVE power creep. Patrick Chapin: "Can we just ban all the cards from the last two years??"